### Merge the application_record and credit decision

In [31]:
import pandas as pd 
app_record = pd.read_csv('../data/application_record.csv')
final_decision = pd.read_csv('../data/credit_decision.csv')
app_record_init = pd.merge(app_record, final_decision, on='ID', how='inner')
app_record_init

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,credit_decision
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,1,0,0,0,Managers,2.0,-1
36453,5149834,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-1
36454,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-1
36455,5150049,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,1,0,0,0,Sales staff,2.0,-1


### Aggregate the income datasheet according to the education

In [32]:
income_train = pd.read_csv('../data/income_train.csv')
income_test = pd.read_csv('../data/income_test.csv')
income = pd.concat([income_train, income_test])
income.drop('income_>50K', axis=1, inplace=True)

def education_categorize(x):
    if x['education'] in ['9th','10th','11th','12th','HS-grad']:
        return 'Secondary / secondary special'
    elif x['education'] in ['1st-4th','5th-6th','7th-8th','Preschool']:
        return 'Lower secondary'
    elif x['education'] in ['Assoc-acdm','Assoc-voc','Some-college']:
        return 'Higher education'
    else:
        return 'Academic degree'
income['education'] = income.apply(education_categorize, axis=1)

import numpy as np
def income_agg(x):
    if x.dtypes == np.int64:
        return x.mean()
    elif x.dtypes == object:
        return x.mode()[0]
income = income.groupby('education').agg(income_agg).reset_index()
income

,education,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,Academic degree,40.906770,Private,186427.490515,13.504990,Married-civ-spouse,Prof-specialty,Husband,White,Male,2783.699811,148.066799,43.297761,United-States
1,Higher education,36.332433,Private,188749.516427,10.361836,Married-civ-spouse,Adm-clerical,Husband,White,Male,586.416742,77.025953,39.454320,United-States
2,Lower secondary,47.083535,Private,209247.739709,3.298426,Married-civ-spouse,Other-service,Husband,White,Male,290.303874,61.390436,38.736077,United-States
3,Secondary / secondary special,38.135474,Private,190745.016961,8.436503,Married-civ-spouse,Craft-repair,Husband,White,Male,504.779882,64.238199,39.552563,United-States


In [33]:
income['capital'] = income['capital-gain'] - income['capital-loss']
income

,education,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,capital
0,Academic degree,40.906770,Private,186427.490515,13.504990,Married-civ-spouse,Prof-specialty,Husband,White,Male,2783.699811,148.066799,43.297761,United-States,2635.633013
1,Higher education,36.332433,Private,188749.516427,10.361836,Married-civ-spouse,Adm-clerical,Husband,White,Male,586.416742,77.025953,39.454320,United-States,509.390789
2,Lower secondary,47.083535,Private,209247.739709,3.298426,Married-civ-spouse,Other-service,Husband,White,Male,290.303874,61.390436,38.736077,United-States,228.913438
3,Secondary / secondary special,38.135474,Private,190745.016961,8.436503,Married-civ-spouse,Craft-repair,Husband,White,Male,504.779882,64.238199,39.552563,United-States,440.541682


### Combine the features of application record and income

In [34]:
app_record_init['NAME_EDUCATION_TYPE'] = app_record_init.apply(lambda x: 'Secondary / secondary special' if x['NAME_EDUCATION_TYPE'] in ['Secondary / secondary special', 'Incomplete higher'] else x['NAME_EDUCATION_TYPE'], axis=1)
app_record = app_record_init[['ID','NAME_EDUCATION_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 
                                   'NAME_FAMILY_STATUS', 'DAYS_EMPLOYED','credit_decision']]
result_df = pd.merge(app_record, income[['education','age', 'workclass', 'occupation', 'capital', 'hours-per-week']], 
                     left_on='NAME_EDUCATION_TYPE', right_on='education', how='left')
result_df.drop('education', axis=1, inplace=True)
result_df

,ID,NAME_EDUCATION_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,DAYS_EMPLOYED,credit_decision,age,workclass,occupation,capital,hours-per-week
0,5008804,Higher education,M,Y,Y,427500.0,Working,Civil marriage,-4542,1,36.332433,Private,Adm-clerical,509.390789,39.454320
1,5008805,Higher education,M,Y,Y,427500.0,Working,Civil marriage,-4542,1,36.332433,Private,Adm-clerical,509.390789,39.454320
2,5008806,Secondary / secondary special,M,Y,Y,112500.0,Working,Married,-1134,-1,38.135474,Private,Craft-repair,440.541682,39.552563
3,5008808,Secondary / secondary special,F,N,Y,270000.0,Commercial associate,Single / not married,-3051,-1,38.135474,Private,Craft-repair,440.541682,39.552563
4,5008809,Secondary / secondary special,F,N,Y,270000.0,Commercial associate,Single / not married,-3051,0,38.135474,Private,Craft-repair,440.541682,39.552563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,Secondary / secondary special,M,Y,Y,315000.0,Working,Married,-2420,-1,38.135474,Private,Craft-repair,440.541682,39.552563
36453,5149834,Higher education,F,N,Y,157500.0,Commercial associate,Married,-1325,-1,36.332433,Private,Adm-clerical,509.390789,39.454320
36454,5149838,Higher education,F,N,Y,157500.0,Pensioner,Married,-1325,-1,36.332433,Private,Adm-clerical,509.390789,39.454320
36455,5150049,Secondary / secondary special,F,N,Y,283500.0,Working,Married,-655,-1,38.135474,Private,Craft-repair,440.541682,39.552563


In [37]:
result_df.to_csv('../data/final_df.csv', index=False)